In [1]:
import numpy as np
import scipy.stats as sc
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [2]:
review_fields = ['review_id', 'user_id', 'business_id', 'stars', 'date']
review_fields2 = ['user_id', 'business_id', 'stars']
business_fields = ['business_id', 'name', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories', 'is_open']
user_fields = ['user_id', 'name', 'review_count', 'yelping_since', 'average_stars']

In [ ]:
prefix = ''#"/Users/Dominic/Downloads/"
business_df = pd.read_csv(prefix + "yelp_business.csv", usecols=business_fields)
review_chunks = pd.read_csv(prefix + "yelp_review.csv",usecols=review_fields,chunksize=500000)
users = pd.read_csv(prefix + "yelp_user.csv",usecols=user_fields)
reviews_raw = pd.read_csv(prefix + "yelp_review.csv",usecols=review_fields2)

In [ ]:
class Sampleable:
    def __init__(self, sample):
        self.sample = self.__mutate(sample)
        assert isinstance(self.sample, pd.core.frame.DataFrame)
        self.sample.dropna()
        self.sample_size = 0
            
    def __mutate(self, sample):
        if isinstance(sample, pd.io.parsers.TextFileReader):
            for s in sample: sample = s; break
        return sample
    
    def getType(self):
        return self.type
    
    def getSample(self, n_samples):
        self.sample_size = n_samples
        rows = np.random.choice(self.sample.index.values, n_samples)
        return self.sample.iloc[rows]
    
    def getOrderedSample(self, n_samples):
        return self.sample.head(n_samples)
    
    def randomMutate(self, new_sample):
        self.sample = self.__mutate(new_sample)
    
businessSampler = Sampleable(business_df)
reviewSampler = Sampleable(review_chunks)
userSampler = Sampleable(users)